In [1]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, ManagedIdentityCredential

In [2]:
subscription_id = "9107822e-69ab-454f-a905-b59c4e15c1af"
resource_group = "AiMethica-ML"
workspace = "AiMethica-AZML"

In [3]:
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [4]:
ws = ml_client.workspaces.get(workspace)
print(ws.location, ":", ws.resource_group)

westeurope : AiMethica-ML


In [5]:
from azure.ai.ml.entities import AmlCompute

cpu_compute_target = "aimethica-cluster-managed"

try:
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"Cluster: {cpu_compute_target} exists."
    )

except Exception:
    print("The cluster doesn't exist.")

Cluster: aimethica-cluster-managed exists.


In [6]:
import os
from azure.ai.ml.entities import Environment

dependencies_dir = "./dependencies"

custom_env_name = "aimethica-anomaly-env"

custom_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for the anomaly detection tasks.",
    conda_file=os.path.join(dependencies_dir, "conda.yaml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)
custom_job_env = ml_client.environments.create_or_update(custom_job_env)

print(
    f"Environment with name {custom_job_env.name} is registered to workspace, the environment version is {custom_job_env.version}"
)

Environment with name aimethica-anomaly-env is registered to workspace, the environment version is 6


In [7]:
cpu_compute_target

'aimethica-cluster-managed'

In [10]:
from azure.ai.ml import command
from azure.ai.ml import Input

env_name = custom_env_name + "@latest"
registered_model_name = "anomaly_document_facturi_export"



job = command(
    inputs=dict(
        input_data_train=Input(
            type="uri_file",
            path="./data/DOCUMENTE_FACTURI_EXPORT_train.csv",
        ),
        input_data_test=Input(
            type="uri_file",
            path="./data/DOCUMENTE_FACTURI_EXPORT_test.csv",
        ),
        epochs = 50,
        batch_size = 16,
    ),
    compute=cpu_compute_target,
    code="./src/",  # location of source code
    command="python anomaly_document_facturi_export.py --input_data_train ${{inputs.input_data_train}} --input_data_test ${{inputs.input_data_test}} --epochs ${{inputs.epochs}} --batch-size ${{inputs.batch_size}}",
    environment=env_name,
    display_name="anomaly_document_facturi_export"
)

In [11]:
returned_job = ml_client.create_or_update(job)
returned_job

Command({'parameters': {}, 'init': False, 'name': 'lucid_circle_20yjr5flyp', 'type': 'command', 'status': 'Starting', 'log_files': None, 'description': None, 'tags': {}, 'properties': {'_azureml.ComputeTargetType': 'amlctrain', '_azureml.ClusterName': 'aimethica-cluster-managed', 'ContentSnapshotId': '08b999e2-2282-4081-9c8d-3d56257bffd6'}, 'print_as_yaml': False, 'id': '/subscriptions/9107822e-69ab-454f-a905-b59c4e15c1af/resourceGroups/AiMethica-ML/providers/Microsoft.MachineLearningServices/workspaces/AiMethica-AZML/jobs/lucid_circle_20yjr5flyp', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/aimethica-general/code/Users/alexandru.popescu/anomaly_DOCUMENTE_FACTURI_EXPORT', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f251972ba60>, 'serialize': <msrest.serialization.Serializer object at 0x7f25197282b0>, 'allowed_keys': {}, 'key_restriction': False, 'logger': <TraceLogger attr_dict (WARNING)>, 'display_name': 'anomaly_document_facturi_export', 'experiment_name': 'anomaly_DOCUMENTE_FACTURI_EXPORT', 'compute': 'aimethica-cluster-managed', 'services': {'Tracking': {'endpoint': 'azureml://westeurope.api.azureml.ms/mlflow/v1.0/subscriptions/9107822e-69ab-454f-a905-b59c4e15c1af/resourceGroups/AiMethica-ML/providers/Microsoft.MachineLearningServices/workspaces/AiMethica-AZML?', 'type': 'Tracking'}, 'Studio': {'endpoint': 'https://ml.azure.com/runs/lucid_circle_20yjr5flyp?wsid=/subscriptions/9107822e-69ab-454f-a905-b59c4e15c1af/resourcegroups/AiMethica-ML/workspaces/AiMethica-AZML&tid=214ea6ae-b24f-4306-b3a4-e32342fa5dfc', 'type': 'Studio'}}, 'comment': None, 'job_inputs': {'input_data_train': {'type': 'uri_file', 'path': 'azureml://datastores/workspaceblobstore/paths/LocalUpload/eeeb42880444c748142fb8e9e87dfb0e/DOCUMENTE_FACTURI_EXPORT_train.csv', 'mode': 'ro_mount'}, 'input_data_test': {'type': 'uri_file', 'path': 'azureml://datastores/workspaceblobstore/paths/LocalUpload/5cec349d3fe611a1acee017444bef38f/DOCUMENTE_FACTURI_EXPORT_test.csv', 'mode': 'ro_mount'}, 'epochs': '50', 'batch_size': '16'}, 'job_outputs': {'default': {'type': 'uri_folder', 'path': 'azureml://datastores/workspaceartifactstore/ExperimentRun/dcid.lucid_circle_20yjr5flyp', 'mode': 'rw_mount'}}, 'inputs': {'input_data_train': <azure.ai.ml.entities._job.pipeline._io.base.NodeInput object at 0x7f2519728250>, 'input_data_test': <azure.ai.ml.entities._job.pipeline._io.base.NodeInput object at 0x7f25197290c0>, 'epochs': <azure.ai.ml.entities._job.pipeline._io.base.NodeInput object at 0x7f251972bac0>, 'batch_size': <azure.ai.ml.entities._job.pipeline._io.base.NodeInput object at 0x7f25197284f0>}, 'outputs': {'default': <azure.ai.ml.entities._job.pipeline._io.base.NodeOutput object at 0x7f25197281f0>}, 'component': CommandComponent({'latest_version': None, 'intellectual_property': None, 'auto_increment_version': True, 'source': 'REMOTE.WORKSPACE.JOB', 'is_anonymous': False, 'auto_delete_setting': None, 'name': 'lucid_circle_20yjr5flyp', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/aimethica-general/code/Users/alexandru.popescu/anomaly_DOCUMENTE_FACTURI_EXPORT', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f251972ba60>, 'serialize': <msrest.serialization.Serializer object at 0x7f2519728070>, 'command': 'python anomaly_document_facturi_export.py --input_data_train ${{inputs.input_data_train}} --input_data_test ${{inputs.input_data_test}} --epochs ${{inputs.epochs}} --batch-size ${{inputs.batch_size}}', 'code': '/subscriptions/9107822e-69ab-454f-a905-b59c4e15c1af/resourceGroups/AiMethica-ML/providers/Microsoft.MachineLearningServices/workspaces/AiMethica-AZML/codes/3ead9918-2dab-4a75-b429-6d91db7457d4/versions/1', 'environment_variables': {}, 'environment': '/subscriptions/9107822e-69ab-454f-a905-b59c4e15c1af/resourceGroups/AiMethic